# Lesson 7 - Creating an A2A Sequential Chain Agent with ADK

Now that you have two active agents (Policy Agent and Research Agent), you will orchestrate them. In this lesson, you will use Google ADK's `SequentialAgent` to create a workflow where a user's query is processed by the Research Agent first, and then the Policy Agent. You will use `RemoteA2aAgent` (which acts as A2A Client) to connect to the servers you started in previous lessons.

![Sequential Workflow](sequential.png)

## 7.1. Start the A2A Servers

First, ensure that your `PolicyAgent` server is running.
- Open Terminal 1 by running the cell below.
- If the agent is still running from the previous lesson, you don't need to do anything.
- If the agent has stopped, type: `uv run a2a_policy_agent.py` (you don't need to go back to the previous lesson).

In [ ]:
import os

from IPython.display import IFrame

url = os.environ.get("DLAI_LOCAL_URL").format(port=8888)
# Terminal 1: uv run a2a_policy_agent.py
IFrame(f"{url}terminals/1", width=800, height=200)

Second, ensure that your Research Agent is running.
- Open Terminal 2 by running the cell below.
- If the agent is still running from the previous lesson, you don't need to do anything.
- If the agent has stopped, type: `uv run a2a_research_agent.py` (you don't need to go back to the previous lesson).

In [ ]:
# Terminal 2: uv run a2a_research_agent.py
IFrame(f"{url}terminals/2", width=800, height=200)

## 7.2. Define the Sequential Workflow

Here you will:
1.  Define two `RemoteA2aAgent` instances. These act as A2A clients that know how to communicate with your running servers via A2A.
2.  Create a `SequentialAgent` named `root_agent`. This agent contains the logic to route the workflow through the sub-agents.
3.  Use an `InMemoryRunner` to execute the flow with a specific prompt.


In [ ]:
import os

from IPython.display import Markdown, display
from dotenv import load_dotenv
from google.adk.agents import SequentialAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
)
from google.adk.runners import InMemoryRunner

import logging
import warnings

logging.disable(level=logging.WARNING)
warnings.filterwarnings("ignore")

In [ ]:
load_dotenv()
host = os.environ.get("AGENT_HOST")
policy_port = os.environ.get("POLICY_AGENT_PORT")
research_port = os.environ.get("RESEARCH_AGENT_PORT")

In [ ]:
policy_agent = RemoteA2aAgent(
    name="policy_agent",
    agent_card=f"http://{host}:{policy_port}",
)
print("\tℹ️", f"{policy_agent.name} initialized")

In [ ]:
health_research_agent = RemoteA2aAgent(
    name="health_research_agent",
    agent_card=f"http://{host}:{research_port}",
)
print("\tℹ️", f"{health_research_agent.name} initialized")

In [ ]:
root_agent = SequentialAgent(
    name="root_agent",
    description="Healthcare Routing Agent",
    sub_agents=[
        health_research_agent,
        policy_agent,
    ],
)
print("\tℹ️", f"{root_agent.name} initialized")

## 7.3. Run the Sequential Chain

The `InMemoryRunner` executes the agent. The prompt will trigger the Research Agent to find general info, and then (sequentially) the Policy Agent to check coverage details.

In [ ]:
prompt = "How can I get mental health therapy?"

**Note:** It takes a few seconds for the output to display.

In [ ]:
print("Running Healthcare Workflow Agent")

runner = InMemoryRunner(root_agent)

for event in await runner.run_debug(prompt, quiet=True):
    if event.is_final_response() and event.content:
        display(Markdown(event.content.parts[0].text))

## 7.4. Resources

- [Google ADK Sequential Agents](https://google.github.io/adk-docs/agents/workflow-agents/sequential-agents/)
- [Equivalent notebook in the course repo](https://github.com/holtskinner/A2AWalkthrough/blob/main/5_ADKSequentialAgent.ipynb)

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download"</em>.</p>
</div>
